In [1]:
import pandas as pd
import random
import math

In [2]:
# load source data
data_df = pd.read_csv("Peru_2019_AudioMoth_Data_Full.csv")

/Users/victoriacyzhang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (1,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# delete samples with short durations
temp_index = data_df[data_df['Duration'] < 60].index
data_df.drop(temp_index, inplace=True)
# delete samples with errors 
data_df = data_df[data_df['Error'].isnull()]

In [4]:
# initialize a list of valid AMs
AMs = {}
for am in data_df.Directory.unique(): # iterate through all possible AMs
    curr = data_df.loc[data_df['Directory'] == am]
    hours = set() # set of hours
    tempDict = {} # dictionary of hour -> list of rows
    for _, row in curr.iterrows(): # iterate through current AM
        value = (pd.to_datetime(row['StartDateTime']).hour) # hour corresponded to current row
        if (math.isnan(value)):
            value = int(row['Comment'][12:14])
        hours.add(value)
        # add row to dictionary
        if value in tempDict.keys():
            tempDict[value].append(row)
        else:
            tempDict[value] = [row]
    # only add current AM if it has enough clips
    if len(hours) == 24:
        AMs[am] = tempDict    

In [5]:
# initialize result list
result = [] 
# pick one random row from each hour
for li in AMs.keys():
    for hour in AMs[li]:
        result.append((random.choice(AMs[li][hour])).to_frame().transpose())

In [6]:
# store result as csv
resultdf = pd.concat(result).reset_index(drop=True)
resultdf.to_csv("result.csv", index=False)

In [7]:
resultdf

,AudioMothCode,AudioMothID,SourceFile,Directory,FileName,FileSize,Encoding,NumChannels,SampleRate,AvgBytesPerSec,BitsPerSample,StartDateTime,Duration,Error,Comment,Artist,FileCreateDate,FileType,FileTypeExtension,MIMEType
0,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190622_155000.WAV,GRABADOR-SDZG-AM-1,20190622_155000.WAV,46080360,1,1,384000,768000,16,22.06.2019 15:50,60.0005,NaN,Recorded at 15:50:00 22/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:40:21-08:00,WAV,WAV,audio/x-wav
1,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190619_162000.WAV,GRABADOR-SDZG-AM-1,20190619_162000.WAV,46080360,1,1,384000,768000,16,19.06.2019 16:20,60.0005,NaN,Recorded at 16:20:00 19/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:26:55-08:00,WAV,WAV,audio/x-wav
2,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190619_173000.WAV,GRABADOR-SDZG-AM-1,20190619_173000.WAV,46080360,1,1,384000,768000,16,19.06.2019 17:30,60.0005,NaN,Recorded at 17:30:00 19/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 03:27:09-08:00,WAV,WAV,audio/x-wav
3,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190628_183000.WAV,GRABADOR-SDZG-AM-1,20190628_183000.WAV,46080360,1,1,384000,768000,16,28.06.2019 18:30,60.0005,NaN,Recorded at 18:30:00 28/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 04:08:03-08:00,WAV,WAV,audio/x-wav
4,AM-1,243B1F055B2BEAB8,GRABADOR-SDZG-AM-1/20190627_192000.WAV,GRABADOR-SDZG-AM-1,20190627_192000.WAV,46080360,1,1,384000,768000,16,27.06.2019 19:20,60.0005,NaN,Recorded at 19:20:00 27/06/2019 (UTC) by Audio...,AudioMoth 243B1F055B2BEAB8,2019:12:20 04:03:42-08:00,WAV,WAV,audio/x-wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,WWF-5,NaN,GRABADOR-WWF-5/5D1A0880.WAV,GRABADOR-WWF-5,5D1A0880.WAV,46080192,1,1,384000,768000,16,NaN,60.0003,NaN,Recorded at 13:20:00 01/07/2019 (UTC) by Audio...,NaN,2019:12:07 15:32:52-08:00,WAV,WAV,audio/x-wav
740,WWF-5,NaN,GRABADOR-WWF-5/5D289260.WAV,GRABADOR-WWF-5,5D289260.WAV,46080192,1,1,384000,768000,16,NaN,60.0003,NaN,Recorded at 14:00:00 12/07/2019 (UTC) by Audio...,NaN,2019:12:07 19:38:36-08:00,WAV,WAV,audio/x-wav
741,WWF-5,NaN,GRABADOR-WWF-5/5D1F6848.WAV,GRABADOR-WWF-5,5D1F6848.WAV,46080192,1,1,384000,768000,16,NaN,60.0003,NaN,Recorded at 15:10:00 05/07/2019 (UTC) by Audio...,NaN,2019:12:07 16:21:06-08:00,WAV,WAV,audio/x-wav
742,WWF-5,NaN,GRABADOR-WWF-5/5D2A0960.WAV,GRABADOR-WWF-5,5D2A0960.WAV,46080192,1,1,384000,768000,16,NaN,60.0003,NaN,Recorded at 16:40:00 13/07/2019 (UTC) by Audio...,NaN,2019:12:07 16:29:18-08:00,WAV,WAV,audio/x-wav
